In [1]:
class Ship:
    def __init__(self, a, b=0, c=0):
        self.a = a
        self.b = b
        self.c = c
    
    # @property
    # def correctness(self):
    #     if 

ship_3x_1 = Ship(11)

In [13]:
# Useful codes
class Point:
    def __init__(self, x = 0, y = 0):
        self.__x = x
        self.__y = y
    
    def __checkValue(x):
        if isinstance(x, int) or isinstance(x, float):
            return True
        else:
            return False
    
    def __getCoordX(self):
        return self.__x
    
    def __setCoordX(self, x):
        if Point.__checkValue(x):
            self.__x = x
        else:
            raise ValueError('Неверный формат')
            
    
    coordX = property(__getCoordX, __setCoordX)

pt = Point("123a", 2)
# pt.coordX = 100
print(pt.coordX)

123a


In [15]:
class Customers:
    def __init__(self, first_name,second_name,city, balance):
        self.first_name = first_name
        self.second_name = second_name
        self.balance = balance
        self.city=city

    def __str__(self):
        return f'''"{self.first_name} {self.second_name}". {self.city}. Баланс: {self.balance} руб.'''

    def get_guest(self):
        return f'{self.first_name} {self.second_name}, г.{self.city}'


costomer_1 = Customers('Иван','Петров','Москва',50)
costomer_2 = Customers('Владимир','Зайцев','Кострома',50)
costomer_3 = Customers('Олеся','Янина','Новосибирск',50)

guest_list=[costomer_1,costomer_2,costomer_3]


for guest in guest_list:
    print(guest.get_guest())

Иван Петров, г.Москва
Владимир Зайцев, г.Кострома
Олеся Янина, г.Новосибирск


In [6]:
grid = [['o' for i in range(6)] for i in range(6)]

def print_grid(L):
    print(*[" ", *[i for i in range(1,7)]], sep=' | ')
    for i in range(6):
        print(i+1, *L[i], sep=' | ')
    return

# print_grid(grid)

us_in = 55
row = us_in // 10
col = us_in % 10
grid[row-1][col-1] = "X"
print_grid(grid)

grid_user = grid.copy()
grid[3][3] = "X"
print_grid(grid_user)
print(id(grid_user))
print(id(grid))

  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | o | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | o | o | o
5 | o | o | o | o | X | o
6 | o | o | o | o | o | o
  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | o | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | X | o | o
5 | o | o | o | o | X | o
6 | o | o | o | o | o | o
1248898175808
1248898175104


In [42]:
class BoardOutError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"{self.value} лежит за пределами игрового поля. Введите двузначное число, каждая цифра которого от 1 до 6, включительно."

class NonEmptyError(Exception):
    def __init__(self,value):
        self.value = value
    
    def __str__(self):
        return f"Выбраная ячейка - {self.value} - непустая. Стрелять можно только в пустые ячейки."

while True:
    try:
        user_shot = int(input("В какую ячейку будем стрелять?"))
        row = user_shot // 10
        col = user_shot % 10

        if row not in range(1,7) or col not in range(1,7):
            err = BoardOutError(user_shot)
            print(err)

        else:
            if grid[row-1][col-1] != 'o':
                err = NonEmptyError(user_shot)
                print(err)
            else:
                grid[row-1][col-1] = 'X'
                print_grid(grid)
                print('Ход компьютера:')
                break

    except Exception:
        print('Необходимо ввести двузначное число!')

В какую ячейку будем стрелять? asd


Необходимо ввести двузначное число!


В какую ячейку будем стрелять? [12.3, 1]


Необходимо ввести двузначное число!


В какую ячейку будем стрелять? 99


99 лежит за пределами игрового поля. Введите двузначное число, каждая цифра которого от 1 до 6, включительно.


В какую ячейку будем стрелять? 66


  | 1 | 2 | 3 | 4 | 5 | 6
1 | o | o | o | o | o | o
2 | o | X | o | o | o | o
3 | o | o | o | o | o | o
4 | o | o | o | o | o | o
5 | o | o | o | o | X | o
6 | o | o | o | o | o | X
Ход компьютера:


In [1]:
class Dot:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        
    def __eq__(self, other):
        return self.row == other and self.col == other

user_shots = []

In [2]:
user_shot = int(input("В какую ячейку будем стрелять?"))
row = user_shot // 10
col = user_shot % 10

shot = Dot(row,col)
user_shots.append(shot)
print(user_shots)

В какую ячейку будем стрелять?55


In [ ]:
print(user_shots)

In [74]:
user_shots[0] == user_shots[1]

False